<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/taxi_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [181]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl


In [182]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 18 13:37 kaggle.json


In [183]:
! kaggle competitions download -c taxi

taxi.zip: Skipping, found more recently modified local copy (use --force to force download)


In [184]:
!unzip taxi.zip

Archive:  taxi.zip
replace submission_form.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission_form.csv     
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [425]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [426]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
      torch.cuda.manual_seed_all(777)

In [557]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [558]:
cab_type = {"Uber":0, "Lyft":1}

train['cab_type']= train['cab_type'].map(cab_type)
test['cab_type']= test['cab_type'].map(cab_type)

In [559]:
summary_mapping = {" Clear ":0," Drizzle ":1," Foggy ":2," Light Rain ":3," Mostly Cloudy ":4," Overcast ":5," Partly Cloudy ":6," Possible Drizzle ":7," Rain ":8}

train['short_summary'] = train['short_summary'].map(summary_mapping)
test['short_summary'] = test['short_summary'].map(summary_mapping)

In [560]:
icon_mapping = {" clear-day ":0," clear-night ":1," cloudy ":2," fog ":3," partly-cloudy-day ":4," partly-cloudy-night ":5," rain ":6}

train['icon'] = train['icon'].map(icon_mapping)
test['icon'] = test['icon'].map(icon_mapping)

In [561]:
x_data = train.loc[0:,"cab_type":"precipIntensityMax"]
y_data = train.loc[0:,"price"]

x_data = np.array(x_data)
y_data = np.array(y_data)
scaler = MinMaxScaler()

x_data = scaler.fit_transform(x_data)
x_train = torch.FloatTensor(x_data).to(device)
y_train = torch.FloatTensor(y_data).to(device).reshape(-1,1)

print(x_train[:5])
print(x_train.shape)
print(y_train[:5])
print(y_train.shape)

tensor([[0.0000, 0.3059, 0.0000, 0.7150, 0.7110, 0.3750, 0.1990, 0.9400, 0.9138,
         0.2811, 0.1391, 0.2777, 1.0000, 0.8546, 0.0042, 1.0000, 0.0000, 0.4073,
         0.9733],
        [0.0000, 0.3059, 0.0000, 0.9321, 0.9423, 0.6250, 0.0000, 0.0000, 0.2241,
         0.7333, 0.4845, 1.0000, 0.3333, 0.7053, 0.2332, 0.9800, 0.5000, 0.3626,
         0.0034],
        [0.0000, 0.3869, 0.0000, 0.3383, 0.3482, 0.5000, 0.0000, 0.0000, 0.7586,
         0.2502, 0.1221, 0.9663, 0.8333, 0.4883, 0.6702, 0.5200, 0.0000, 0.1233,
         0.0000],
        [0.0000, 0.3869, 0.0000, 0.4962, 0.4387, 0.5000, 0.0000, 0.0000, 0.5690,
         0.5381, 0.4382, 0.9980, 0.8333, 0.5408, 0.1955, 0.7500, 0.0000, 0.7260,
         0.0000],
        [0.0000, 0.3869, 0.0000, 0.6990, 0.7443, 0.2500, 0.0000, 0.0000, 0.9828,
         0.0412, 0.0068, 0.0000, 0.5000, 0.8637, 0.2676, 1.0000, 0.0000, 0.2283,
         0.0027]], device='cuda:0')
torch.Size([2809, 19])
tensor([[32.5000],
        [11.0000],
        [27.0000],
  

In [562]:
x_test = test.loc[0:,"cab_type":]
x_test = np.array(x_test)
x_test = scaler.transform(x_test)
x_test = torch.FloatTensor(x_test).to(device)

x_test[:3]

tensor([[1.0000, 0.0196, 0.0000, 0.6116, 0.5542, 0.5000, 0.0000, 0.0000, 0.5172,
         0.5643, 0.3146, 1.0000, 0.8333, 0.6117, 0.7141, 0.7200, 0.0000, 0.3142,
         0.8746],
        [1.0000, 0.0196, 0.0000, 0.6440, 0.5593, 1.0000, 0.8977, 1.0000, 0.9655,
         0.7924, 0.4212, 0.4383, 1.0000, 0.8090, 0.3346, 1.0000, 0.0000, 0.1982,
         0.8910],
        [1.0000, 0.0196, 0.0000, 0.5069, 0.4613, 0.0000, 0.0000, 0.0000, 0.6379,
         0.4729, 0.2449, 1.0000, 0.1667, 0.5771, 0.0883, 0.0300, 0.0000, 0.4228,
         0.7293]], device='cuda:0')

In [617]:
learning_rate = 0.005
training_epochs = 1000
batch_size = 50

In [618]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [619]:
linear1=torch.nn.Linear(19,8,bias=True)
linear2=torch.nn.Linear(8,8,bias=True)
linear3=torch.nn.Linear(8,8,bias=True)
linear4=torch.nn.Linear(8,8,bias=True)
linear5=torch.nn.Linear(8,1,bias=True)
dropout=torch.nn.Dropout(p=0.3)
relu= torch.nn.LeakyReLU()


In [620]:
torch.nn.init.kaiming_uniform_(linear1.weight)
torch.nn.init.kaiming_uniform_(linear2.weight)
torch.nn.init.kaiming_uniform_(linear3.weight)
torch.nn.init.kaiming_uniform_(linear4.weight)
torch.nn.init.kaiming_uniform_(linear5.weight)


Parameter containing:
tensor([[ 6.7234e-01,  1.4234e-04, -6.7970e-01, -8.3873e-01, -2.3578e-01,
         -7.9360e-01, -3.4571e-01,  3.3311e-01]], requires_grad=True)

In [621]:
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5).to(device)

In [622]:
loss = torch.nn.MSELoss(reduction='mean').to(device)
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [623]:
total_batch = len(data_loader)

for epoch in range(training_epochs+1):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch
        
    if(epoch%10 == 0):    
          print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))
          print(cost.item())

print('Learning finished')

Epoch: 0000 cost = 456.762969971
511.95147705078125
Epoch: 0010 cost = 175.547973633
159.09535217285156
Epoch: 0020 cost = 139.810806274
207.80758666992188
Epoch: 0030 cost = 138.359008789
92.18795013427734
Epoch: 0040 cost = 127.528419495
238.23805236816406
Epoch: 0050 cost = 125.144752502
88.90643310546875
Epoch: 0060 cost = 125.177108765
109.08124542236328
Epoch: 0070 cost = 121.224479675
70.35060119628906
Epoch: 0080 cost = 125.595977783
102.43356323242188
Epoch: 0090 cost = 114.940170288
210.2611846923828
Epoch: 0100 cost = 121.289047241
153.71966552734375
Epoch: 0110 cost = 122.138313293
51.7901496887207
Epoch: 0120 cost = 118.985527039
121.42807006835938
Epoch: 0130 cost = 123.767715454
87.10724639892578
Epoch: 0140 cost = 125.642509460
200.54232788085938
Epoch: 0150 cost = 115.303924561
177.70579528808594
Epoch: 0160 cost = 116.117546082
65.66770935058594
Epoch: 0170 cost = 117.373939514
88.8910903930664
Epoch: 0180 cost = 109.500450134
65.95841979980469
Epoch: 0190 cost = 123.

In [624]:
x_test = test.loc[0:,"cab_type":]
x_test = np.array(x_test)
x_test = scaler.transform(x_test)
x_test = torch.FloatTensor(x_test).to(device)

x_test[:3]


tensor([[1.0000, 0.0196, 0.0000, 0.6116, 0.5542, 0.5000, 0.0000, 0.0000, 0.5172,
         0.5643, 0.3146, 1.0000, 0.8333, 0.6117, 0.7141, 0.7200, 0.0000, 0.3142,
         0.8746],
        [1.0000, 0.0196, 0.0000, 0.6440, 0.5593, 1.0000, 0.8977, 1.0000, 0.9655,
         0.7924, 0.4212, 0.4383, 1.0000, 0.8090, 0.3346, 1.0000, 0.0000, 0.1982,
         0.8910],
        [1.0000, 0.0196, 0.0000, 0.5069, 0.4613, 0.0000, 0.0000, 0.0000, 0.6379,
         0.4729, 0.2449, 1.0000, 0.1667, 0.5771, 0.0883, 0.0300, 0.0000, 0.4228,
         0.7293]], device='cuda:0')

In [625]:
model(x_train)

tensor([[16.8975],
        [16.1150],
        [17.2573],
        ...,
        [61.2075],
        [43.0294],
        [36.0015]], device='cuda:0', grad_fn=<AddmmBackward>)

In [626]:
with torch.no_grad():
    model.eval()
    x_test=np.array(x_test.cpu())
    x_test=torch.from_numpy(x_test).float().to(device)
    predict=model(x_test)

In [627]:
correct_prediction = predict.cpu().numpy().reshape(-1,1)
result = pd.read_csv('submission_form.csv')

In [628]:
for i in range(len(predict)):
      result['price'][i]=predict[i].item()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [629]:
result['price'] = result['price'].astype(int)

In [630]:
result.to_csv('submit.csv', index=False)

In [631]:
!kaggle competitions submit -c taxi -f submit.csv -m "14010974 이기택"

100% 601/601 [00:00<00:00, 657B/s]
Successfully submitted to 택시비 예측